<a href="https://colab.research.google.com/github/EjbejaranosAI/EmotionUnify/blob/main/Multibench_fast_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pliang279/MultiBench.git
%cd MultiBench

Cloning into 'MultiBench'...
remote: Enumerating objects: 6931, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 6931 (delta 65), reused 121 (delta 60), pack-reused 6789
Receiving objects: 100% (6931/6931), 51.06 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (4251/4251), done.
/content/MultiBench


In [2]:
import os
import torch
import sys
import gdown

# Create the directory if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Download the file to the directory
url = 'https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU'
output = 'data/mosi_raw.pkl'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU
To: /content/MultiBench/data/mosi_raw.pkl
100%|██████████| 357M/357M [00:02<00:00, 130MB/s]


'data/mosi_raw.pkl'

In [3]:

# Import the associated dataloader for affect datasets, which MOSI is a part of.
from datasets.affect.get_data import get_dataloader

# Create the training, validation, and test-set dataloaders.
traindata, validdata, testdata = get_dataloader(
    '/content/MultiBench/data/mosi_raw.pkl', robust_test=False, max_pad=True, data_type='mosi', max_seq_len=50)

In [4]:

# Here, we'll import several common modules should you want to mess with this more.
from unimodals.common_models import GRU, MLP, Sequential, Identity

# As this example is meant to be simple and easy to train, we'll pass in identity
# functions for each of the modalities in MOSI:
encoders = [Identity().cuda(), Identity().cuda(), Identity().cuda()]

In [5]:

# Import a fusion paradigm, in this case early concatenation.
from fusions.common_fusions import ConcatEarly  # noqa

# Initialize the fusion module
fusion = ConcatEarly().cuda()


In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [7]:
import torch
print(torch.version.cuda)

11.8


In [8]:
head = Sequential(GRU(409, 512, dropout=True, has_padding=False,
                  batch_first=True, last_only=True), MLP(512, 512, 1)).cuda()

In [10]:
!pip install memory_profiler

In [11]:

# Standard supervised learning training loop
from training_structures.Supervised_Learning import train, test

# For more information regarding parameters for any system, feel free to check out the documentation
# at multibench.readthedocs.io!
train(encoders, fusion, head, traindata, validdata, 100, task="regression", optimtype=torch.optim.AdamW,
      is_packed=False, lr=1e-3, save='mosi_ef_r0.pt', weight_decay=0.01, objective=torch.nn.L1Loss())

print("Testing:")
model = torch.load('mosi_ef_r0.pt').cuda()
test(model, testdata, 'affect', is_packed=False,
     criterion=torch.nn.L1Loss(), task="posneg-classification", no_robust=True)


Epoch 0 train loss: tensor(1.3353, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 0 valid loss: 1.3834203481674194
Saving Best
Epoch 1 train loss: tensor(1.3278, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 1 valid loss: 1.3926174640655518
Epoch 2 train loss: tensor(1.3206, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 valid loss: 1.391062617301941
Epoch 3 train loss: tensor(1.3234, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 valid loss: 1.3722001314163208
Saving Best
Epoch 4 train loss: tensor(1.3202, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 valid loss: 1.3868460655212402
Epoch 5 train loss: tensor(1.3177, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 5 valid loss: 1.3763071298599243
Epoch 6 train loss: tensor(1.3176, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 6 valid loss: 1.378089189529419
Epoch 7 train loss: tensor(1.3165, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 7 valid loss: 1.3772833347320557
Epoch 8 train loss: tensor(1.3186, device='cuda:0', grad_f